In [1]:
import numpy as np 

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from keras.utils.np_utils import to_categorical

from sklearn.utils import shuffle
from process_data import process_document
import json

e:\users\ranet\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = json.load(open('toxic_data2.json', encoding='UTF8'))

x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

In [3]:
print(len(x_train), 'len of x train')
print(len(y_train), 'len of y train')

print(len(x_test), 'len of x test')
print(len(y_test), 'len of y test')

print(x_train[0])
print(y_train[0])

print(x_test[0])
print(y_test[0])

print("Number of 100: {}".format(y_train.count("100"))) #100 is improper
print("Number of 105: {}".format(y_train.count("105")))

print("Number of 100 in val: {}".format(y_test.count("100"))) # 100 is improper
print("Number of 105 in val: {}".format(y_test.count("105")))

X = x_train
X_val = x_test

y = [0 if int(y) == 100 else 1 for y in y_train]
y_val = [0 if int(y) == 100 else 1 for y in y_test]


from statistics import mean
lengths = [len(x_ex.split(' ')) for x_ex in X]
lengths.sort()
print("Mean lengths of comments: {}".format(mean(lengths)))
print("Mean lengths of comments with the last 25 longest comments removed: {}"
      .format(mean(lengths[:-25])))

9944 len of x train
9944 len of y train
2487 len of x test
2487 len of y test
Kohalikud venelased ehk saavadki kuidagi oma eraisiku tasemel ja nÃ¤gemusega hakkama aga kui riik oma kohaolu ignoreerib siis kohalikud hakkavad otsima neid kes neid EI ignereeri ja ulatab nii vajaliku abikÃ¤e.... Loodus tÃ¼hja kohta ei salli . Kui Eesti jaoks on Ida-Virumaa kui regioon pohh-nahh siis ehk siis silmad avanevad kui korraga juhtmed seinast vÃ¤ljatÃµmmatass? ja kus ã¼lejã¤ã¤nud eesti tã¶ã¶tajaskond on kes teenib miinimumi lã¤hedast palka ? istute mã¶kuna oma peldikus ja vahite ukseaugust pealt ? vã¤hemalt ametiã¼hingud peaksid kãµik nagu ã¼ks mees vã¤ljas olema aga nemad soojendavad kotte kuskil kontoora nurgas ja teevad valitsusega isiklikke diile .... aivo peterson 09 09 62.181.220.227
105
HÃ¤lvik on hÃ¤lvik.  tånis 13 09 194.150.65.37
105
Number of 100: 5358
Number of 105: 4586
Number of 100 in val: 1348
Number of 105 in val: 1139
Mean lengths of comments: 50.466311343523735
Mean lengths of c

In [4]:
vocab_size = 50000
seq_len = 60

tokenizer = Tokenizer(num_words=seq_len,
                                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n12345687890',
                                   lower=True,
                                   split=" ",
                                   char_level=False,
                                   oov_token=None)


tokenizer.fit_on_texts(X_val + X)
print(len(tokenizer.word_index))

X_tokenized = tokenizer.texts_to_sequences(X)
X_val_tokenized = tokenizer.texts_to_sequences(X_val)

X_tokenized = pad_sequences(X_tokenized, maxlen=seq_len)
X_val_tokenized = pad_sequences(X_val_tokenized, maxlen=seq_len)

print(X_val[0])
print(X_val_tokenized[0])
print(X[0])
print(X_tokenized[0])


print(y[0:3])
y_oh = to_categorical(y, 2)
print(y_oh[0:3])
print(y_val[0:3])
y_val_oh = to_categorical(y_val, 2)
print(y_val_oh[0:3])

75162
HÃ¤lvik on hÃ¤lvik.  tånis 13 09 194.150.65.37
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
Kohalikud venelased ehk saavadki kuidagi oma eraisiku tasemel ja nÃ¤gemusega hakkama aga kui riik oma kohaolu ignoreerib siis kohalikud hakkavad otsima neid kes neid EI ignereeri ja ulatab nii vajaliku abikÃ¤e.... Loodus tÃ¼hja kohta ei salli . Kui Eesti jaoks on Ida-Virumaa kui regioon pohh-nahh siis ehk siis silmad avanevad kui korraga juhtmed seinast vÃ¤ljatÃµmmatass? ja kus ã¼lejã¤ã¤nud eesti tã¶ã¶tajaskond on kes teenib miinimumi lã¤hedast palka ? istute mã¶kuna oma peldikus ja vahite ukseaugust pealt ? vã¤hemalt ametiã¼hingud peaksid kãµik nagu ã¼ks mees vã¤ljas olema aga nemad soojendavad kotte kuskil kontoora nurgas ja teevad valitsusega isiklikke diile .... aivo peterson 09 09 62.181.220.227
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 10  1  9  5 10  7 58 12 

In [5]:
embed_dim = 64
lstm_out = 32

model = Sequential()
model.add(Embedding(vocab_size, embed_dim, input_length=seq_len, dropout=0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout_U=0.4, dropout_W=0.4)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

e:\users\ranet\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
e:\users\ranet\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, dropout=0.4, recurrent_dropout=0.4)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 64)            3200000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 3,224,962
Trainable params: 3,224,962
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
num_epochs = 50
batch_size = 128

model.fit(X_tokenized, y_oh, epochs=num_epochs, batch_size=batch_size, verbose=1, validation_data=(X_val_tokenized, y_val_oh))

Train on 9944 samples, validate on 2487 samples
Epoch 1/50
9944/9944 [==============================] - 19s 2ms/step - loss: 0.6837 - acc: 0.5513 - val_loss: 0.6751 - val_acc: 0.5762
Epoch 2/50
9944/9944 [==============================] - 15s 2ms/step - loss: 0.6792 - acc: 0.5624 - val_loss: 0.6723 - val_acc: 0.5830
Epoch 3/50
9944/9944 [==============================] - 15s 2ms/step - loss: 0.6768 - acc: 0.5686 - val_loss: 0.6714 - val_acc: 0.5786
Epoch 4/50
9944/9944 [==============================] - 16s 2ms/step - loss: 0.6756 - acc: 0.5706 - val_loss: 0.6734 - val_acc: 0.5862
Epoch 5/50
9944/9944 [==============================] - 15s 2ms/step - loss: 0.6745 - acc: 0.5739 - val_loss: 0.6722 - val_acc: 0.5690
Epoch 6/50
9944/9944 [==============================] - 15s 2ms/step - loss: 0.6727 - acc: 0.5762 - val_loss: 0.6732 - val_acc: 0.5653
Epoch 7/50
9944/9944 [==============================] - 15s 2ms/step - loss: 0.6722 - acc: 0.5790 - val_loss: 0.6719 - val_acc: 0.5758
Epoch 8

KeyboardInterrupt: 